In [1]:
import cv2
from matplotlib import pyplot as plt
import numpy as np 
import imutils
import easyocr
from pathlib import Path
import pandas as pd

C:\Users\Acer\AppData\Local\Temp\ipykernel_3600\1017591537.py:7: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
folder = Path(r"C:\Users\Acer\Documents\เอกสาร\GitHub\Sci-project-m5\2024-01-31")
original_excel = pd.read_excel("data\cdi-data-scan1266.xlsx")

In [3]:
global score_table
score_table = pd.DataFrame(original_excel[['ID', 'Pt']])

print(score_table)

      ID  Pt
0      0   5
1      1   8
2      2  15
3      3   8
4      4  13
..   ...  ..
475  543  23
476  544   6
477  545  28
478  546  13
479  547  32

[480 rows x 2 columns]


In [4]:
i = score_table['Pt'][score_table['ID']==2].index
# print(i)
print(score_table.loc[i[0], 'Pt'])

15


In [5]:
def labelClass(ID):
    D = "Depressed"
    noD = "No-Depressed"

    i = score_table['Pt'][score_table['ID']==ID].index
    score = score_table.loc[i[0], 'Pt']
    
    if score >= 15:
        return D
    else:
        return noD

In [6]:
def cropped(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    bfilter = cv2.bilateralFilter(gray, 15, 20, 20)
    edged = cv2.Canny(bfilter, 30, 200)

    keypoint = cv2.findContours(edged.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    contours = imutils.grab_contours(keypoint)
    contours = sorted(contours, key=cv2.contourArea, reverse=True)[:10]

    mask = np.zeros(gray.shape, np.uint8)
    location = None

    for contour in contours:
        # print(i)
        approx = cv2.approxPolyDP(contour, 10, True)
        if len(approx) == 4:
            max_x = max(approx[:,:,1])
            min_x = min(approx[:,:,1])
            max_y = max(approx[:,:,0])
            min_y = min(approx[:,:,0])

            x_lenght = max_x-min_x
            y_lenght = max_y-min_y

            if x_lenght > 500 and y_lenght > 500:
                location = approx
                break
        
    if location is not None:
        new_image = cv2.drawContours(mask, [location], 0, 255, -1)
        new_image = cv2.bitwise_and(img, img, mask=mask)

        (x, y) = np.where(mask==255)
        (x1, y1) = (np.min(x), np.min(y))
        (x2, y2) = (np.max(x), np.max(y))
        cropped_image = gray[x1:x2+1, y1:y2+1]
        return 1, cropped_image

    return 0, gray

In [7]:
def detectID(path):
    result = []
    img = cv2.imread(path)
    img = img[1600: , 100:500]
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    reader = easyocr.Reader(['en'])
    result = reader.readtext(gray)

    return result

In [8]:
def denoise(img):
    lower = 0  
    upper = 240 

    mask = cv2.inRange(img, lower, upper)

    result = cv2.bitwise_and(img, img, mask=mask)
    # plt.imshow(cv2.cvtColor(result, cv2.COLOR_BGR2RGB))
    # cv2.imwrite('test-inrange-denoising.jpg', result)
    return result

In [13]:
i = 0

for jpg_file_path in list(folder.glob('*.jpg')):
    result = detectID(str(jpg_file_path))
    # print(result)

    original_img = cv2.imread(str(jpg_file_path))
    check, cropped_img = cropped(original_img)
    result_img = denoise(cropped_img)

    if len(result) > 0:
        Id = result[0][1].rjust(3, '0')
        label = labelClass(int(result[0][1]))
        if check:
            cv2.imwrite(f'data_label\\{label}\\{Id}.jpg', result_img)
        else:
            cv2.imwrite(f'data_label\\test-img\\incomplete-{Id}.jpg', result_img)
    else:
        cv2.imwrite(f'data_label\\noID-{i}.jpg', result_img)
        i += 1
        # print("no Id path : ", jpg_file_path)
    # cv2.imwrite(f'data_label\\test-{i}.jpg', result_img)
    # i += 1



error: OpenCV(4.9.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\color.cpp:196: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'
